In [5]:
from datetime import timedelta

from dydx_v4_client.indexer.rest.indexer_client import IndexerClient  # type: ignore
from dydx_v4_client.network import make_mainnet  # type: ignore
from sqlalchemy import create_engine  # type: ignore


from locast.candle_fetcher.dydx.api_fetcher.dydx_v4_fetcher import DydxV4Fetcher
from locast.candle_fetcher.dydx.candle_fetcher.dydx_v4_candle_fetcher import (
    DydxV4CandleFetcher,
)
from locast.store_manager.store_manager import StoreManager

from locast.candle.candle_utility import CandleUtility as cu
from locast.candle.dydx.dydx_resolution import DydxResolution
from locast.candle.exchange import Exchange
from locast.candle_storage.sql.sqlite_candle_storage import SqliteCandleStorage

MAINNET = make_mainnet(  # type: ignore
    rest_indexer="https://indexer.dydx.trade/",
    websocket_indexer="wss://indexer.dydx.trade/v4/ws",
    node_url="dydx-ops-rpc.kingnodes.com:443",
)


async def dydx_v4_example() -> None:
    # Create store manager
    storage = SqliteCandleStorage(create_engine("sqlite:///locast.db"))  # type: ignore
    fetcher = DydxV4CandleFetcher(DydxV4Fetcher(IndexerClient(MAINNET.rest_indexer)))  # type: ignore
    manager = StoreManager(candle_fetcher=fetcher, candle_storage=storage)

    # Define desired candle cluster parameters for eth and btc
    exchange = Exchange.DYDX_V4
    resolution = DydxResolution.ONE_MINUTE
    eth = "ETH-USD"
    btc = "BTC-USD"

    # Define start date for candle cluster creation
    now = cu.normalized_now(resolution)  # Now, rounded down to nearest resolution tick
    start_date = now - timedelta(seconds=resolution.seconds * 10000)  # 10K candles back

    # Check for existing clusters
    eth_info = await storage.get_cluster_info(exchange, eth, resolution)
    btc_info = await storage.get_cluster_info(exchange, btc, resolution)

    # If clusters not present, create (download from exchange & store in database)
    if not eth_info.head:
        await manager.create_cluster(eth, resolution, start_date)
    if not btc_info.head:
        await manager.create_cluster(btc, resolution, start_date)

    # Check if clusters are up to date and update if not
    if not eth_info.is_uptodate:
        await manager.update_cluster(exchange, eth, resolution)
    if not btc_info.is_uptodate:
        await manager.update_cluster(exchange, btc, resolution)

    eth_info = await storage.get_cluster_info(exchange, eth, resolution)
    btc_info = await storage.get_cluster_info(exchange, btc, resolution)

    print(f"ETH candles: {eth_info.amount}.")
    print(f"BTC candles: {btc_info.amount}.")


await dydx_v4_example()

Batch #0 size: 283
Candles left: 0
Batch #0 size: 283
Candles left: 0
ETH candles: 10293.
BTC candles: 10293.
